# Make 3D descriptors
### Requirements
dscribe (only MacOS and Linux)

ASE

Pandas
### Represent database structures as ASE atoms

In [3]:
from ase.io import read
from ase.build import molecule
from ase import Atoms

from pathlib import Path

In [4]:
structures = [read(str(path)) for path in Path("./coordinates").glob('**/*.xyz')]

### Get all species present

In [5]:
species = set()
for structure in structures:
    species.update(structure.get_chemical_symbols())

### Define some descriptors

In [6]:
from dscribe.descriptors import SOAP

In [15]:
soap1 = SOAP(
    species=species,
    periodic=False,
    rcut=5,#cutoff in Angstroms
    nmax=1,#number of radial basis functions
    lmax=1,#maximum degree of spherical harmonics
    average="outer",
    sparse=False
)
soap2 = SOAP(
    species=species,
    periodic=False,
    rcut=3,
    nmax=1,
    lmax=1,
    average="outer",
    sparse=False
)

In [16]:
feature_vectors_1 = soap1.create(structures, n_jobs=1)
feature_vectors_2 = soap2.create(structures, n_jobs=1)

### Save as csv files

In [17]:
import pandas as pd
import os
import numpy as np
import sys

In [18]:
index_values = [os.fsdecode(file)[:-4] for file in os.listdir(os.fsencode("./coordinates"))]
column_values = ["SOAP 1","SOAP 2"]
df = pd.DataFrame(data = [[feature_vectors_1[i],feature_vectors_2[i]] for i in range(len(feature_vectors_1))], 
                  index = index_values, 
                  columns = column_values)

In [19]:
np.set_printoptions(threshold=sys.maxsize)
df.to_csv("soap_descriptors.csv")